In [1]:
import pandas as pd
import time
import numpy as np
import os
import random
from datetime import datetime   
import math
from tqdm.notebook  import tqdm
from shutil import copytree

pd.options.mode.chained_assignment  = None

In [2]:
# Define the path for all of the folders.
ED_FOLDER_PATH = "E:/AI-VR dataset/MIMIC-IV ED"
CLINICAL_FOLDER_PATH = "E:/AI-VR dataset/MIMIC-IV Clinical Database"
CXR_FOLDER_PATH = "E:/AI-VR dataset/MIMIC-CXR-JPG/physionet.org/files/mimic-cxr-jpg/2.0.0"
EYEGAZE_FOLDER_PATH = "E:/AI-VR dataset/eye-gaze-data-for-chest-x-rays-1.0.0"
REFLACX_FOLDER_PATH = "E:/AI-VR dataset/reflacx-reports-and-eye-tracking-data-for-localization-of-abnormalities-in-chest-x-rays-1.0.0/reflacx-reports-and-eye-tracking-data-for-localization-of-abnormalities-in-chest-x-rays-1.0.0"
# XAMI_MIMIC_PATH = "E:/AI-VR dataset/XAMI-MIMIC"
XAMI_MIMIC_PATH = "./XAMI-MIMIC"

In [6]:
cxr_df_with_stay_id_only = pd.read_csv("cxr_meta_with_stay_id_only.csv", index_col=0)

In [10]:
cxr_df_with_stay_id_only

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,StudyDateTime,stay_id,in_eye_gaze,in_reflacx,in_ed,in_core
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect,2.180051e+13,33258284,False,False,True,True
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect,2.180051e+13,33258284,False,False,True,True
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect,2.180063e+13,38112554,False,False,True,True
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect,2.180063e+13,38112554,False,False,True,True
6,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,10000032,56699142,CHEST (PORTABLE AP),AP,3056,2544,21800805,234424.765,CHEST (PORTABLE AP),antero-posterior,NaN,2.180081e+13,35968195,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377097,431c5823-ea54bba2-f21af1d4-45887776-9de7edd1,19999287,53255195,CHEST (PORTABLE AP),AP,2544,3056,21970803,194419.125,CHEST (PORTABLE AP),antero-posterior,Erect,2.197080e+13,37929752,False,False,True,True
377104,3fcd0406-9b111603-feae7033-96632b3a-111333e5,19999733,57132437,CHEST (PA AND LAT),PA,3056,2544,21520708,224550.171,CHEST (PA AND LAT),postero-anterior,Erect,2.152071e+13,30940569,False,False,True,True
377105,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c,19999733,57132437,CHEST (PA AND LAT),PA,3056,2544,21520708,224550.171,CHEST (PA AND LAT),postero-anterior,Erect,2.152071e+13,30940569,False,False,True,True
377106,58c403aa-35ff8bd9-73e39f54-8dc9cc5d-e0ec3fa9,19999733,57132437,CHEST (PA AND LAT),LATERAL,3056,2544,21520708,224550.171,CHEST (PA AND LAT),lateral,Erect,2.152071e+13,30940569,False,False,True,True


In [14]:
all_subjects_in_cxr = list(set(cxr_df_with_stay_id_only["subject_id"]))
all_subjects_in_cxr.sort()

In [72]:
for subject_id in tqdm(all_subjects_in_cxr):

    subject_match = cxr_df_with_stay_id_only[cxr_df_with_stay_id_only['subject_id'] == subject_id]
    all_stay_ids = list(set(subject_match["stay_id"]))

    for stay_id in all_stay_ids:
        stay_match = subject_match[subject_match["stay_id"] == stay_id]
        all_study_ids = list(set(stay_match['study_id']))

        for study_id in all_study_ids:
            ### CXR-jpg
            source_path = f"{CXR_FOLDER_PATH}/files/p{str(subject_id)[:2]}/p{subject_id}/s{study_id}"
            save_folder_path = f"{XAMI_MIMIC_PATH}/patient_{subject_id}/stay_{stay_id}/CXR-JPG"
            os.makedirs(save_folder_path, exist_ok=True)
            destination_path = f"{save_folder_path}/study_{study_id}"
            copytree(source_path, destination_path)

            ## CXR-meta
            match_cxr_meta = cxr_df_with_stay_id_only[cxr_df_with_stay_id_only['study_id'] == study_id]
            match_cxr_chexpert = cxr_chexpert_df[cxr_chexpert_df["study_id"] == study_id]
            match_cxr_negbio = cxr_negbio_df[cxr_negbio_df["study_id"] == study_id]
            match_cxr_split = cxr_split_df[cxr_split_df["study_id"] == study_id]

            match_cxr_meta.to_csv(f"{destination_path}/cxr_chexpert.csv")
            match_cxr_chexpert.to_csv(f"{destination_path}/cxr_chexpert.csv")
            match_cxr_negbio.to_csv(f"{destination_path}/cxr_negbio.csv")
            match_cxr_split.to_csv(f"{destination_path}/cxr_split.csv") 


            ### Dicom


  0%|          | 0/29639 [00:00<?, ?it/s]